### Program 5:
#### Objective:
Implement a tiny version of the UNet image segmentation architecture using the PyTorch framework, and train it on the VOCSegmentation dataset.
Perform the following steps:
- Preprocess data
- Define TinyUNet architecture
- Define model train function
- Train model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader, Subset

In [ ]:
class TinyUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=21):
        super(TinyUNet, self).__init__()
        def conv_block(in_channels, out_channels):
            return nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                 nn.ReLU(),
                                 nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                                 nn.ReLU())
        self.encoder1 = conv_block(in_channels, 16)
        self.encoder2 = conv_block(16, 32)
        self.encoder3 = conv_block(32, 64)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bottleneck = conv_block(64, 128)
        self.upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder3 = conv_block(128, 64)
        self.upconv2 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.decoder2 = conv_block(64, 32)
        self.upconv1 = nn.ConvTranspose2d(32, 16, kernel_size=2, stride=2)
        self.decoder1 = conv_block(32, 16)
        self.conv_final = nn.Conv2d(16, out_channels, kernel_size=1)
    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        bottleneck = self.bottleneck(self.pool(enc3))
        dec3 = self.upconv3(bottleneck)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return self.conv_final(dec1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load VOC Segmentation dataset
train_dataset = VOCSegmentation(root='./data', year='2012', image_set='train', download=True, transform=transform, target_transform=transform)
test_dataset = VOCSegmentation(root='./data', year='2012', image_set='val', download=True, transform=transform, target_transform=transform)
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))

In [ ]:
# Define DataLoader
train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)
model = TinyUNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            outputs = model(data).to(device)
            loss = criterion(outputs, target.squeeze(1).long())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * data.size(0)
        
        avg_train_loss = train_loss / len(train_loader)

        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data).to(device)
                
                loss = criterion(outputs, target.squeeze(1).long())
     
                test_loss += loss.item() * data.size(0)
        
        avg_test_loss = test_loss / len(test_loader)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

In [ ]:
train(model, optimizer, criterion, 10)

Epoch [1/10], Train Loss: 22.0729, Test Loss: 10.9271
Epoch [2/10], Train Loss: 5.7279, Test Loss: 3.2615
Epoch [3/10], Train Loss: 2.7386, Test Loss: 2.4970
Epoch [4/10], Train Loss: 2.4597, Test Loss: 2.4428
Epoch [5/10], Train Loss: 2.3781, Test Loss: 2.1537
Epoch [6/10], Train Loss: 2.1615, Test Loss: 2.0441
Epoch [7/10], Train Loss: 2.0990, Test Loss: 2.0370
Epoch [8/10], Train Loss: 2.0379, Test Loss: 2.0097
Epoch [9/10], Train Loss: 2.0023, Test Loss: 1.9855
Epoch [10/10], Train Loss: 2.0700, Test Loss: 2.0159
